In [1]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time
import copy
from unicodedata import normalize

In [2]:
class TrieNode(object):

    
    def __init__(self, char: str):   # __init__ é um método especial para fazer construtores
        self.char = char   # caractere do nodo atual
        self.filhos = []   # nodos filhos
        self.pFinalizada = False   # se é o último nodo e a palavra terminou
        self.indices = []   # lista vazia para nodos que não são término de palavra
        ### não é um índice único e sim uma lista, pois podem existir títulos de música repetidos com mais de um artista


#Definindo função que localiza uma string na arvore
def findString(select, dados, raiz, palavra: str) -> (bool, []):
    """
      1. Se a palavra existe e tem algum índice associado, retorna verdadeiro e a lista de índices dela
      2. Se a palavra não existe, retorna falso e uma lista com sugestões de palavras
      
      Sendo o parâmetro select utilizada para indicar se a busca é por artista ou música (0 música, 1 artista)
    """
    palavra = formataString(palavra)   # formata string de entrada para o padrão (todas as letras maiúsculas sem acento)
    nodo = raiz
    stringSugerida = ""  # guarda os caracteres já encontrados para sugerir os próximos
    
    if not raiz.filhos:   # se o nodo raiz não tiver nenhum filho, trivial, retorna falso
        return False, []
    for char in palavra:
        charNaoEncontrado = True
        for filho in nodo.filhos:
            if filho.char == char:
                charNaoEncontrado = False   # assinala que o char foi encontrado
                stringSugerida = stringSugerida+char
                nodo = filho   # passa iteração para o nodo filho
                break
        
        if charNaoEncontrado:   # chama a função que procura sugestões e retorna a tupla (False, lista de sugestões)
            return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
    # Caso passe por todos os caracteres sem retornar false, então significa que a palavra foi encontrada
    # Resta saber se aquele nodo é um nodo final com um índice associado
    if nodo.pFinalizada:
        return True, nodo.indices
    else:
        return False, sugereStrings(select, dados, nodo, stringSugerida, 15)
    
# Função que formata string desconsiderando maiúsculas e minúsculas e que remove acentos
def formataString(string):
    string = normalize('NFKD', string).encode('ASCII', 'ignore').decode('ASCII')
    return string.upper()

def sugereStrings(select, dados, nodo, string, maxSugestoes):   # onde select igual a 1 significa que se procura um artista e False, música
        listaDeSugestoes = []   # cria lista vazia para adicionarmos as sugestões
        procuraSugestoes(nodo, string, listaDeSugestoes)   # chama a função que percorre a trie procurando sugestões
        
        itera = copy.deepcopy(listaDeSugestoes)   # utilizado para iteração na lista que está sendo modificada enquanto o loop itera
        
        # substitui os indices pelos títulos
        for indice in itera:
            listaDeSugestoes.remove(indice)   # remove o índice 
            if select:
                listaDeSugestoes.append(dados[indice]['Artista']) 
            else:
                listaDeSugestoes.append(dados[indice]['Titulo']) 
            
        
        listaDeSugestoes = sorted(listaDeSugestoes)   # ordena a lista alfabeticamente
        return listaDeSugestoes[:maxSugestoes]   # retorna apenas os n elementos da lista definidos por maxSugestoes

    
# função que dado um nodo, uma parte correta de uma string (que existe na TRIE), uma lista de sugestoes, retorna uma lista de sugestões
def procuraSugestoes(nodo, string, listaDeSugestoes):
        
        if nodo.pFinalizada:   
            listaDeSugestoes.append(nodo.indices[0])   # adiciona nova sugestão na lista
        
        for filho in nodo.filhos:            
            procuraSugestoes(filho, string+filho.char, listaDeSugestoes)   # recursão para cada filho 

        return        

In [3]:
with (open('database.bin', 'rb')) as openfile:
    database = pickle.load(openfile)
        
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

#Abrindo os arquivos com os tops de artistas
with (open('topArtistas.bin', 'rb')) as openfile:
    topArtistas = pickle.load(openfile)
        
with (open('topMusicas.bin', 'rb')) as openfile:
    topMusicas = pickle.load(openfile)
        
#Abrindo os arquivos com as arvores Trie      
with (open('trieMusicas.bin', 'rb')) as openfile:
    trieMusicas = pickle.load(openfile)
        
with (open('trieArtistas.bin', 'rb')) as openfile:
    trieArtistas = pickle.load(openfile)

In [4]:
#Retorna o artista mais popular e quantas semanas ele ficou na billboard
def comparaRelevArtista(artista1, artista2):
    indicesArtista1 = findString(1, database, trieArtistas, artista1) #retorna os indices do arquivo principal com registros do artista
    indicesArtista2 = findString(1, database, trieArtistas, artista2) #retorna os indices do arquivo principal com registros do artista
    
    #variaveis com o numero de semanas que cada artista ficou na hot 100
    semanas1 = 0
    semanas2 = 0
    
    if (indicesArtista1[0] == False and indicesArtista2[0] == False): #testa se as duas entradas sao invalidas
        return None
    
    elif (indicesArtista1[0] == False or indicesArtista2[0] == False): #testa se achou os 2 artistas
        if indicesArtista1[0] == False: #se nao achou o primeiro devolve o segundo
            for i in indicesArtista2[1]:
                semanas2 += database[i]['Semanas'] #Adiciona o numero de semanas     
            return(artista2, semanas2)
        else: #se nao achou o segundo devolve o primeiro
            for i in indicesArtista1[1]:
                semanas1 += database[i]['Semanas'] #Adiciona o numero de semanas     
            return(artista1, semanas1)
    
    #compara os 2
    else:
        for i in indicesArtista1[1]:
            semanas1 += database[i]['Semanas'] #Adiciona o numero de semanas 
        for i in indicesArtista2[1]:
            semanas2 += database[i]['Semanas'] #Adiciona o numero de semanas 
            
        if(semanas1 > semanas2): #artista1 é mais relevante que o segundo
            return(artista1, semanas1)
        elif(semanas2 > semanas1): #artista2 é mais relevante que o primeiro
            return(artista2, semanas2)
        else: #trata o empate
            pontos1 = 0
            pontos2 = 0
            
            for i in indicesArtista1[1]:
                pontos1 += database[i]['Pontos'] #Adiciona o numero de semanas 
            for i in indicesArtista2[1]:
                pontos2 += database[i]['Pontos'] #Adiciona o numero de semanas 
            if (pontos1 > pontos2):
                return(artista1, semanas1)
            else:
                return(artista1, semanas1)

In [5]:
print(comparaRelevArtista('Foo Fighters', 'Elton John'))
print(comparaRelevArtista('Daives', 'Colombelli'))

('Elton John', 875)
None


In [6]:
#Retorna a musica mais popular e quantas semanas ele ficou na billboard
def comparaRelevMusica(musica1, artista1, musica2, artista2):
    musica1Inds = findString(1, database, trieMusicas, musica1) #retorna os indices do arquivo principal com registros da musica
    musica2Inds = findString(1, database, trieMusicas, musica2) #retorna os indices do arquivo principal com registros da musica
        
    achou = 0 #se for 0 retorna que nao encontrou
    
    #------------------------------------------------------------------------------
    #Testa se alguma das musicas nao é valida
    #------------------------------------------------------------------------------
    if (musica1Inds[0] == False and musica2Inds[0] == False): #testa se as 2 entradas sao invalidas
        return None
    
    elif (musica1Inds[0] == False or musica2Inds[0] == False): #testa se achou os 2 artistas
        if musica1Inds[0] == False: #se nao achou o primeiro devolve o segundo
            for x in musica2Inds[1]:
                if (database[x]['Artista'] == artista2):
                    achou = 1
                    return(musica2, database[x]['Semanas'])
        else: #se nao achou o segundo devolve o primeiro
            for x in musica1Inds[1]:
                if (database[x]['Artista'] == artista1):
                    achou = 1
                    return(musica1, database[x]['Semanas'])
    
    #as 2 musicas sao validas
    else:
        for x in musica1Inds[1]:
                if (database[x]['Artista'] == artista1):
                    achou = 1
                    semanas1 = database[x]['Semanas']
                    indice1 = x
        for x in musica2Inds[1]:
                if (database[x]['Artista'] == artista2):
                    achou = 1
                    semanas2 = database[x]['Semanas']
                    indice2 = x
                    
        if(semanas1 > semanas2):
            return(musica1, semanas1)
        elif(semanas2 > semanas1):
            return(musica2, semanas2)
        else: #mesmo numero de semanas
            if(database[indice1]['Pontos'] > database[indice2]['Pontos']):
                return(musica1, semanas1)
            else:
                return(musica2, semanas2)

In [10]:
print(comparaRelevMusica('Borderline', 'Madonna','Crazy For You','Madonna'))
print(comparaRelevMusica('Lucky Star', 'Madonna','Crazy For You','Madonna'))
print(comparaRelevMusica('I Guess That\'s Why They Call It The Blues', 'Elton John','Crazy For You','Madonna'))
print(comparaRelevMusica('Olar', 'Madonna','Crazy For','Madonna'))

TypeError: comparaRelevMusica() missing 3 required positional arguments: 'artista1', 'musica2', and 'artista2'

## A versão abaixo é a versão final utilizada no arquivo .py

In [30]:
def comparaRelevMusica(trieMusicas, trieArtistas, database, musica1, artista1, musica2, artista2):
    musica1Inds = findString(0, database, trieMusicas, musica1) #retorna os indices do arquivo principal com registros da musica
    musica2Inds = findString(0, database, trieMusicas, musica2) #retorna os indices do arquivo principal com registros da musica

    artista1 = findString(1, database, trieArtistas, artista1)
    artista2 = findString(1, database, trieArtistas, artista2)
    
    achouArtista1 = 0 #flags que servem para garantir que os 2 artistas inseridos pelo usuario estejam no banco de dados
    achouArtista2 = 0
    
    if artista1[0]:   # se o artista passado se encontra na trie
        achouArtista1 = 1
        artista1 = database[artista1[1][0]]['Artista']   # pega o nome do jeito que tá na base de dados

    # repete o mesmo processo para o artista 2
    if artista2[0]:
        achouArtista2 = 1
        artista2 = database[artista2[1][0]]['Artista']
        
        
    if(achouArtista1 == 0 and achouArtista2 == 0):
        return None
    elif(achouArtista1 == 0):
        musica1Inds = (False,False) #como python nao permite alterar o valor de uma tupla temos que criar uma tupla nova e substituir na variavel
    else:
        musica2Inds = (False,False)

    achou = 0 #se for 0 retorna que nao encontrou

    #------------------------------------------------------------------------------
    #Testa se alguma das musicas nao é valida
    #------------------------------------------------------------------------------
    if (musica1Inds[0] == False and musica2Inds[0] == False): #testa se as 2 entradas sao invalidas
        return None

    elif (musica1Inds[0] == False or musica2Inds[0] == False): #testa se achou os 2 artistas
        if musica1Inds[0] == False: #se nao achou o primeiro devolve o segundo
            for x in musica2Inds[1]:
                if (database[x]['Artista'] == artista2):
                    achou = 1
                    return(database[x]['Titulo'], database[x]['Semanas'])
        else: #se nao achou o segundo devolve o primeiro
            for x in musica1Inds[1]:
                if (database[x]['Artista'] == artista1):
                    achou = 1
                    return(database[x]['Titulo'], database[x]['Semanas'])

    #as 2 musicas sao validas
    else:
        for x in musica1Inds[1]:
                if (database[x]['Artista'] == artista1):
                    achou = 1
                    semanas1 = database[x]['Semanas']
                    indice1 = x
        for x in musica2Inds[1]:
                if (database[x]['Artista'] == artista2):
                    achou = 1
                    semanas2 = database[x]['Semanas']
                    indice2 = x

        if(semanas1 > semanas2):
            return(database[musica1Inds[1][0]]['Titulo'], semanas1)
        elif(semanas2 > semanas1):
            return(database[musica2Inds[1][0]]['Titulo'], semanas2)
        else: #mesmo numero de semanas
            if(database[indice1]['Pontos'] > database[indice2]['Pontos']):
                return(database[musica1Inds[1][0]]['Titulo'], semanas1)
            else:
                return(database[musica2Inds[1][0]]['Titulo'], semanas2)

In [34]:
print(comparaRelevMusica(trieMusicas, trieArtistas, database,'mine', 'taylor swifot','mean','taylor swlift'))

None


In [8]:
musica1Inds = findString(1, database, trieMusicas, 'I Love You') #retorna os indices do arquivo principal com registros da musica
print(musica1Inds[1][0])

567


In [9]:
print(database[4806])
print(database[5005])
print(database[4859])
teste = findString(1, database, trieArtistas, 'Elton John')[1]
for x in teste:
    print(database[x])

{'Artista': 'Madonna', 'Titulo': 'Borderline', 'Ano': 1984, 'Pontos': 1641, 'Peak': 91, 'Semanas': 30}
{'Artista': 'Madonna', 'Titulo': 'Crazy For You', 'Ano': 1985, 'Pontos': 1519, 'Peak': 100, 'Semanas': 21}
{'Artista': 'Madonna', 'Titulo': 'Lucky Star', 'Ano': 1984, 'Pontos': 1134, 'Peak': 97, 'Semanas': 16}
{'Artista': 'Elton John', 'Titulo': 'Your Song', 'Ano': 1971, 'Pontos': 785, 'Peak': 93, 'Semanas': 14}
{'Artista': 'Elton John', 'Titulo': 'Rocket Man', 'Ano': 1972, 'Pontos': 1095, 'Peak': 95, 'Semanas': 15}
{'Artista': 'Elton John', 'Titulo': 'Honky Cat', 'Ano': 1972, 'Pontos': 723, 'Peak': 93, 'Semanas': 10}
{'Artista': 'Elton John', 'Titulo': 'Levon', 'Ano': 1972, 'Pontos': 546, 'Peak': 77, 'Semanas': 10}
{'Artista': 'Elton John', 'Titulo': 'Daniel', 'Ano': 1973, 'Pontos': 1184, 'Peak': 99, 'Semanas': 15}
{'Artista': 'Elton John', 'Titulo': 'Crocodile Rock', 'Ano': 1973, 'Pontos': 1181, 'Peak': 100, 'Semanas': 17}
{'Artista': 'Elton John', 'Titulo': "Saturday Night's Alrigh